In [20]:
#!pip install tabpfn
!pip install lightgbm

In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import classification_report
import numpy as np
from sklearn.tree import DecisionTreeClassifier
df = pd.read_csv('/kaggle/input/edgeiiotset-cyber-security-dataset-of-iot-iiot/Edge-IIoTset dataset/Selected dataset for ML and DL/ML-EdgeIIoT-dataset.csv', low_memory=False)



In [22]:

def group_attacks10(df):
    df["Attack_type"] = df["Attack_type"].map(lambda v: "Anomalous" if v == "Ransomware" or v == "Password" or v == "SQL_injection" or v == "Uploading"
                                                   else v)
#def group_attacks10(df):
#    df["Attack_type"] = df["Attack_type"].map(lambda v: "Inf.Gathe" if v == "Fingerprinting" or v == "Port_Scanning" or v == "Vulnerability_scanner"
#                                                   else ( "Injection" if v == "XSS" or v == "Uploading"
#                                                         else ("Malware" if v == "Backdoor" or v == "Password" or v == "Ransomware" else v)))

print(df['Attack_type'].value_counts())

Attack_type
Normal                   24301
DDoS_UDP                 14498
DDoS_ICMP                14090
Ransomware               10925
DDoS_HTTP                10561
SQL_injection            10311
Uploading                10269
DDoS_TCP                 10247
Backdoor                 10195
Vulnerability_scanner    10076
Port_Scanning            10071
XSS                      10052
Password                  9989
MITM                      1214
Fingerprinting            1001
Name: count, dtype: int64


In [23]:
from sklearn.utils import shuffle

drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 

         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",

         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",

         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)

df.dropna(axis=0, how='any', inplace=True)

df.drop_duplicates(subset=None, keep="first", inplace=True)

df = shuffle(df)

df.isna().sum()

print(df['Attack_type'].value_counts())

Attack_type
Normal                   24101
DDoS_UDP                 14498
DDoS_ICMP                13096
DDoS_HTTP                10495
SQL_injection            10282
DDoS_TCP                 10247
Uploading                10214
Vulnerability_scanner    10062
Password                  9972
Backdoor                  9865
Ransomware                9689
XSS                       9543
Port_Scanning             8921
Fingerprinting             853
MITM                       358
Name: count, dtype: int64


In [24]:
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing

def encode_text_dummy(df, name):

    dummies = pd.get_dummies(df[name])

    for x in dummies.columns:

        dummy_name = f"{name}-{x}"

        df[dummy_name] = dummies[x]

    df.drop(name, axis=1, inplace=True)

encode_text_dummy(df,'http.request.method')

encode_text_dummy(df,'http.referer')

encode_text_dummy(df,"http.request.version")

encode_text_dummy(df,"dns.qry.name.len")

encode_text_dummy(df,"mqtt.conack.flags")

encode_text_dummy(df,"mqtt.protoname")

encode_text_dummy(df,"mqtt.topic")

In [25]:
f = ['http.request.version-0.0', 'http.request.method-0.0',
 'http.request.version-0', 'http.request.method-0', 'http.referer-0.0',
 'http.referer-0', 'tcp.flags', 'tcp.flags.ack', 'tcp.checksum', 'tcp.ack_raw',
 'icmp.checksum', 'udp.stream', 'tcp.connection.syn', 'icmp.seq_le', 'tcp.ack',
 'tcp.connection.rst', 'tcp.seq', 'tcp.len', 'http.request.version-HTTP/1.1',
 'http.request.method-GET', 'http.response', 'http.content_length',
 'tcp.connection.fin', 'dns.qry.name', 'http.request.version-HTTP/1.0']

df10 = df.copy()
group_attacks10(df10)

X = df10[f]
X10 = df10[f]
y = df10["Attack_type"]
y10 = df10["Attack_type"]
scale = StandardScaler()
#X[X.columns] = scale.fit_transform(X)
X10[X10.columns] = scale.fit_transform(X10)
from sklearn.model_selection import train_test_split
train = 250
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
X10_train, X10_test, y10_train, y10_test = train_test_split(X10, y10, test_size=0.3, random_state=0, stratify=y10)

from sklearn.preprocessing import LabelEncoder


lgb_model = LGBMClassifier(n_jobs=-1,random_state=0)
lgb_model.fit(X_train,y_train)
l_pred = lgb_model.predict(X_test)

print(classification_report(y_test,l_pred))


rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
r_pred = rf_model.predict(X_test)

print(classification_report(y_test,r_pred))

ad_model = AdaBoostClassifier(random_state=0)
ad_model.fit(X_train,y_train)
ad_pred = ad_model.predict(X_test)


dt_model = DecisionTreeClassifier()
dt_model.fit(X_train,y_train)
dt_pred = dt_model.predict(X_test)

/tmp/ipykernel_34/1430981174.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X10[X10.columns] = scale.fit_transform(X10)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2355
[LightGBM] [Info] Number of data points in the train set: 106537, number of used features: 25
[LightGBM] [Info] Start training from score -1.332367
[LightGBM] [Info] Start training from score -2.736247
[LightGBM] [Info] Start training from score -2.674336
[LightGBM] [Info] Start training from score -2.452882
[LightGBM] [Info] Start training from score -2.698168
[LightGBM] [Info] Start training from score -2.351117
[LightGBM] [Info] Start training from score -5.184331
[LightGBM] [Info] Start training from score -6.050795
[LightGBM] [Info] Start training from score -1.842896
[LightGBM] [Info] Start training from score -2.836711
[LightGBM] [Info] Start training from score -2.716458
[LightGBM] [Info] Start training from score -2.76937

tb_model = TabPFNClassifier()
tb_model.fit(X10_train[0:1000],y10_train[0:1000])
t_pred = tb_model.predict(X10_test)

print(classification_report(y10_test,t_pred))

from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score


precision = precision_score(y_test, x_pred, average="macro")
recall = recall_score(y_test, x_pred, average="macro")
f1 = f1_score(y_test, x_pred, average="macro")
accuracy = accuracy_score(y_test,r_pred)
print("Xgbbost accuracy:",accuracy)
print("Xgboost Precision:", precision)
print("Xgboost Recall:", recall)
print("Xgboost F1 Score:", f1)

In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score


precision = precision_score(y_test, r_pred, average="macro")
recall = recall_score(y_test, r_pred, average="macro")
f1 = f1_score(y_test, r_pred, average="macro")
accuracy = accuracy_score(y_test,r_pred)
print("RF accuracy:",accuracy)
print("RF Precision:", precision)
print("RF Recall:", recall)
print("RF F1 Score:", f1)

RF accuracy: 0.9691846076348584
RF Precision: 0.9569344031177205
RF Recall: 0.9469468445436434
RF F1 Score: 0.9511782498315347


from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score

precision = precision_score(y10_test, t_pred, average="macro")
recall = recall_score(y10_test, t_pred, average="macro")
f1 = f1_score(y10_test, t_pred, average="macro")
accuracy = accuracy_score(y10_test,t_pred)
print("Tab Accuracy:",accuracy)
print("Tab Precision:", precision)
print("Tab Recall:", recall)
print("Tab F1 Score:", f1)

In [27]:
precision = precision_score(y_test, l_pred, average="macro")
recall = recall_score(y_test, l_pred, average="macro")
f1 = f1_score(y_test, l_pred, average="macro")
accuracy = accuracy_score(y_test,l_pred)
print("Light accuracy:",accuracy)
print("Light Precision:", precision)
print("Light Recall:", recall)
print("Light F1 Score:", f1)

Light accuracy: 0.5244092073851815
Light Precision: 0.44178339262717065
Light Recall: 0.3878482021300113
Light F1 Score: 0.37887519980532364


In [28]:
precision = precision_score(y_test, ad_pred, average="macro")
recall = recall_score(y_test, ad_pred, average="macro")
f1 = f1_score(y_test, ad_pred, average="macro")
accuracy = accuracy_score(y_test,ad_pred)
print("Ada accuracy:",accuracy)
print("Ada Precision:", precision)
print("Ada Recall:", recall)
print("Ada F1 Score:", f1)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Ada accuracy: 0.42806456558400313
Ada Precision: 0.14340100724744398
Ada Recall: 0.25
Ada F1 Score: 0.1716772982810814


In [29]:
precision = precision_score(y_test, dt_pred, average="macro")
recall = recall_score(y_test, dt_pred, average="macro")
f1 = f1_score(y_test, dt_pred, average="macro")
accuracy = accuracy_score(y_test,dt_pred)
print("Ada accuracy:",accuracy)
print("Ada Precision:", precision)
print("Ada Recall:", recall)
print("Ada F1 Score:", f1)

Ada accuracy: 0.9639063492411135
Ada Precision: 0.9422674761510993
Ada Recall: 0.9401280324571317
Ada F1 Score: 0.9411677898169236
